#### This notebook demonstrates the use of the EOP post-processing algorithm for bias mitigation.


In [8]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from eq_odds_postprocessing import EqOddsPostprocessing
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso
from random import sample

## Huangrui's Dataset 

In [9]:
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
    
class BankDataset(StandardDataset):
    """financial-inclusion-in-africa dataset.
    """

    def __init__(self, path, label_name='y', favorable_classes=[1],  
                 protected_attribute_names=['age'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_drop=[],
                 features_to_keep=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        """See :obj:`RegressionDataset` for a description of the arguments."""
        

        df = pd.read_csv(path)
        df["cons.conf.idx"] = -df["cons.conf.idx"]
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)

        super(BankDataset, self).__init__(
            df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [10]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [11]:
experiments_info = {}
bef_experiments_info = {}
budget = 0.1
for K in range(1, 6):
    dataset_orig_train= BankDataset(path="./Huangrui/bank/bank_train{}.csv".format(K))
    dataset_orig_test= BankDataset(path="./Huangrui/bank/bank_test{}.csv".format(K))
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)
    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/bank'+str(K)+'_age_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred>0.5
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)


    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test.features
    y_test = dataset_orig_test.labels
    y_test_pred = lmod.predict(X_test)
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)
    dataset_orig_test_pred.labels = (y_test_pred>0.5).reshape(-1,1)
    #load EOP Model
    EOP = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=random_seed)
    #fit EOP model
    EOP = EOP.fit(dataset_orig_train, dataset_orig_train_pred)
    print("Fitted the EOP model")
    # get the EOP predictions for test (Transform the test set)
    dataset_transf_test_pred = EOP.predict(dataset_orig_test_pred)
    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}
    bef_experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels), "Equal opportunity difference": metric_test_bef["Equal opportunity difference"]}

Fitted the EOP model
Balanced accuracy = 0.6318
Statistical parity difference = 0.0740
Disparate impact = 2.2255
Average odds difference = 0.0585
Equal opportunity difference = 0.1017
Theil index = 0.1053
Balanced accuracy = 0.6236
Statistical parity difference = -0.0013
Disparate impact = 0.9792
Average odds difference = -0.0778
Equal opportunity difference = -0.1637
Theil index = 0.1076
K = 1, budget = 0.1
The Error for the test dataset is 0.1179
The Equal opportunity difference for the test dataset is -0.1637
Fitted the EOP model
Balanced accuracy = 0.6624
Statistical parity difference = 0.1273
Disparate impact = 2.9953
Average odds difference = 0.1444
Equal opportunity difference = 0.2313
Theil index = 0.0952
Balanced accuracy = 0.6558
Statistical parity difference = 0.0212
Disparate impact = 1.2273
Average odds difference = -0.0147
Equal opportunity difference = -0.0242
Theil index = 0.1018
K = 2, budget = 0.1
The Error for the test dataset is 0.1281
The Equal opportunity differen

In [12]:
bef_experiments_info

{'K = 1, budget = 0.1': {'Error': 0.11595866819747416,
  'Equal opportunity difference': 0.10165155980648388},
 'K = 2, budget = 0.1': {'Error': 0.10511643161692358,
  'Equal opportunity difference': 0.23134759976865238},
 'K = 3, budget = 0.1': {'Error': 0.10774024270252541,
  'Equal opportunity difference': 0.2281614528101803},
 'K = 4, budget = 0.1': {'Error': 0.10888816005247622,
  'Equal opportunity difference': 0.24292527821939586},
 'K = 5, budget = 0.1': {'Error': 0.11005412497949811,
  'Equal opportunity difference': 0.24330042313117067}}

In [13]:
experiments_info

{'K = 1, budget = 0.1': {'Error': 0.11792684927013285,
  'Equal opportunity difference': -0.1636545626424957},
 'K = 2, budget = 0.1': {'Error': 0.12807477861593966,
  'Equal opportunity difference': -0.02421438210911897},
 'K = 3, budget = 0.1': {'Error': 0.19776976057723844,
  'Equal opportunity difference': 0.046361346765641576},
 'K = 4, budget = 0.1': {'Error': 0.15808461790751066,
  'Equal opportunity difference': 0.043163751987281396},
 'K = 5, budget = 0.1': {'Error': 0.17041167787436445,
  'Equal opportunity difference': 0.13790550070521862}}

# For budget =0.01, do it seperately

In [14]:
experiments_info = {}

In [27]:
budget = 0.01
K = 1
dataset_orig_train= BankDataset(path="./Huangrui/bank/bank_train{}.csv".format(K))
dataset_orig_test= BankDataset(path="./Huangrui/bank/bank_test{}.csv".format(K))
#only use the budget% of the training data
dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=True)
# Lasso linear classifier and predictions
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()
lmod = pickle.load(open('experiments/bank'+str(K)+'_age_bmodel.pkl','rb'))["clf"]

y_train_pred = lmod.predict(X_train)

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred>0.5
sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)


dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels
y_test_pred = lmod.predict(X_test)
dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)
dataset_orig_test_pred.labels = (y_test_pred>0.5).reshape(-1,1)
#load EOP Model
EOP = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=random_seed)
#fit EOP model
EOP = EOP.fit(dataset_orig_train, dataset_orig_train_pred)
print("Fitted the EOP model")
# get the EOP predictions for test (Transform the test set)
dataset_transf_test_pred = EOP.predict(dataset_orig_test_pred)
metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#自己计算error, 不是balanced accuracy！！！
print("K = {}, budget = {}".format(K, budget))
print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}
bef_experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels), "Equal opportunity difference": metric_test_bef["Equal opportunity difference"]}

Fitted the EOP model
Balanced accuracy = 0.6318
Statistical parity difference = 0.0740
Disparate impact = 2.2255
Average odds difference = 0.0585
Equal opportunity difference = 0.1017
Theil index = 0.1053
Balanced accuracy = 0.6291
Statistical parity difference = 0.0633
Disparate impact = 2.0474
Average odds difference = 0.0250
Equal opportunity difference = 0.0200
Theil index = 0.1061
K = 1, budget = 0.01
The Error for the test dataset is 0.1169
The Equal opportunity difference for the test dataset is 0.02002


In [28]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.11694275873380351,
  'Equal opportunity difference': 0.020018906745259413},
 'K = 2, budget = 0.01': {'Error': 0.138570022958347,
  'Equal opportunity difference': -0.14524773472141891},
 'K = 3, budget = 0.01': {'Error': 0.3832404066907183,
  'Equal opportunity difference': -0.3524986744432662},
 'K = 4, budget = 0.01': {'Error': 0.427189242374549,
  'Equal opportunity difference': -0.3728139904610493},
 'K = 5, budget = 0.01': {'Error': 0.14843365589634247,
  'Equal opportunity difference': -0.15747531734837802}}